In [1]:
import requests
import os
import json
from pprint import pprint
from dotenv import load_dotenv

#export 'BEARER_TOKEN'='<your_bearer_token>' >>> initially tried that syntax but didn't work, variables returned none
load_dotenv()  # take environment variables from .env.

bearer_token = os.environ.get("BEARER_TOKEN")
qb_user_token = os.environ.get("QB_USER_TOKEN")

#get request arguments query_params, headers and search_url
query_params = {
    'query': '#QuickBase OR #NoCode OR #LowCode OR #QBCommunitySummit',
    # https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference 
    'tweet.fields':'author_id,created_at,entities',
    # https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users-by-username-username
    'expansions':'author_id',
    'user.fields': 'username,id',
    #Twitter pagination from 10 to 100
    'max_results': 100,
}

# 2 endpoints available, "all" and "recent". All is only available for academic and research. I only have access to recent which gives me the most recent 7 days
# https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-all
# https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent
search_url = "https://api.twitter.com/2/tweets/search/recent"

headers = {
    "Authorization": f"Bearer {bearer_token}"
}

In [13]:
#token to request the next page due to Twitter API pagination >> max 100/page min 10/page
#https://developer.twitter.com/en/docs/twitter-api/pagination
next_token = None

#dictionary to map userids to usernames
users = {}

#list of tweets
tweets = []

for i in range(5): #looping to get X number of tweets (more than 100)
    
    # next_token only used starting the second request 
    if i > 0:
        query_params['next_token'] = next_token
    
    # sending get request to twitter API
    response = requests.get(search_url, headers=headers, params=query_params)
    page = response.json()
    next_token = page['meta']['next_token']
    
    # upserting usernames to users dictionary
    for user in page['includes']['users']:
        userid = user['id']
        username = user['username']
        users[userid] = username
        
    # Reshaping the twitter data with the proper formatting for the Quickbase API
    for tweet in page['data']:
        tweet_id = tweet['id']
        tweet_text = tweet['text']
        tweet_created_at = tweet['created_at']
        tweet_author_id = tweet['author_id']
        
        #extracting tags and exception handling for when no hashtags are returned >> get 'hashtags', [] otherwise 
        tweet_hashtags = [hashtag['tag'] for hashtag in tweet['entities'].get('hashtags', [])]
        
        #twitter strips data after 140th character. resulting in incomplete records (in some cases not including the tags we're looking for),
        #although the original tweet includes the hashtags, hence them showing in the response in the first place!
        
        #filtering out records that don't include the tags in the response
        if "nocode" in tweet_hashtags or 'QuickBase' in tweet_hashtags or 'LowCode' in tweet_hashtags or'QBCommunitySummit' in tweet_hashtags:
            tweets.append({
                
                #'record ID#'
                '10': { #'11' for the copy of tweets table in QB
                    "value": tweet_id
                },
                
                #'Hashtags'
                '6': {
                    "value": tweet_hashtags
                },
                
                #'Twitter_Username'
                '7':{
                    "value": users.get(tweet_author_id, None)
                },
                
                #'Tweet_Content'
                '8': {
                    "value":tweet_text
                },
                
                #'Date'
                '9':{
                    "value": tweet_created_at.split("T")[0] # for date field in QB
                    #"value": tweet_created_at for date/time field in QB
                },
            })
len(tweets)

125

In [12]:
#get request arguments query_params, headers and 
query_params2 = {
    'query': '#QuickBase OR #NoCode OR #LowCode OR #QBCommunitySummit',
    # https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference 
    'tweet.fields':'author_id,created_at',
    # https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users-by-username-username
    'expansions':'author_id',
    'user.fields': 'username,id',
    #Twitter pagination from 10 to 100
    'max_results': 10,
}

# 2 endpoints available, "all" and "recent". All is only available for academic and research. I only have access to recent which gives me the most recent 7 days
# https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-all
# https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent
search_url2 = "https://api.twitter.com/2/tweets/search/recent"

headers2 = {
    "Authorization": f"Bearer {bearer_token}"
}

response = requests.get(search_url2, headers=headers2, params=query_params2).json()

#response['data']
list(map(lambda x: (x['text'],x['id']), response['data']))


##https://twitter.com/i/web/status/1508213112306081809

[('===Notes/Domino開発tipsの概要紹介===\n\n3/25に予告したとおり、本日（3/28）分のtipsが配信されました。お客様におきましてはNotes DBで内容をご確認いただけます。\n\n関連ブログは以下のリンクを確認してください\n\n#tips #dominoforever #カレンダー #日付 #lowcode\nhttps://t.co/RTIKDIHkxn',
  '1508243824472358912'),
 ('RT @VeilleCyber3: Artificial intelligence is everywhere now. \nhttps://t.co/0fTbgb6TUi\n\n#AIEthics #MachineLearning #AI #Python #DeepLearning…',
  '1508243204558266377'),
 ('RT @VeilleCyber3: Artificial intelligence is everywhere now. \nhttps://t.co/0fTbgb6TUi\n\n#AIEthics #MachineLearning #AI #Python #DeepLearning…',
  '1508241910145564672'),
 ('RT @hjpatelspace: Check new Updates on our Space🚀\n\nRead Details : https://t.co/K43cJhe49M\n\n#HJPatel \n#100DaysOfCode #WomenWhoCode #python #…',
  '1508239939594510338'),
 ('RT @hjpatelspace: Check new Updates on our Space🚀\n\nRead Details : https://t.co/K43cJhe49M\n\n#HJPatel \n#100DaysOfCode #WomenWhoCode #python #…',
  '1508239774255095822'),
 ('RT @VeilleCyber3: Why #African #banks are investing in #AI \nhttps

In [4]:
#formatting JSON for Quickbase post request
body = {
    #"to": "br9e6snt2", #copy of tweets table in QB
    "to": "br9btjupe",
    "data": tweets,
}

jsontweets = json.dumps(body)
#jsontweets

In [5]:
qb_headers = {
  	'QB-Realm-Hostname': 'team.quickbase.com',
    #same thing here, storing it here while working on the assignment..
	'Authorization': f'QB-USER-TOKEN {qb_user_token}'
}

r = requests.post(
        'https://api.quickbase.com/v1/records', 
        headers = qb_headers, 
        json = body
    )

upload = r.json()
upload

#https://twitter.com/Youssef60079013/status/1508214468467204096
#https://twitter.com/i/web/status/1508216889922752500
#tweeted once with all hashtags to test

{'data': [],
 'metadata': {'createdRecordIds': [153,
   154,
   155,
   156,
   157,
   158,
   159,
   160,
   161,
   162,
   163,
   164,
   165,
   166,
   167,
   168,
   169,
   170,
   171,
   172,
   173,
   174,
   175,
   176,
   177,
   178,
   179,
   180,
   181,
   182,
   183,
   184,
   185,
   186,
   187,
   188,
   189,
   190,
   191,
   192,
   193,
   194,
   195,
   196,
   197,
   198,
   199,
   200,
   201,
   202,
   203,
   204,
   205,
   206,
   207,
   208,
   209,
   210,
   211,
   212,
   213,
   214,
   215,
   216,
   217,
   218,
   219,
   220,
   221,
   222,
   223,
   224,
   225,
   226,
   227,
   228,
   229,
   230,
   231,
   232,
   233,
   234,
   235,
   236,
   237,
   238,
   239,
   240,
   241,
   242,
   243,
   244,
   245,
   246,
   247,
   248,
   249,
   250,
   251,
   252,
   253,
   254,
   255,
   256,
   257],
  'totalNumberOfRecordsProcessed': 139,
  'unchangedRecordIds': [5,
   8,
   9,
   16,
   20,
   21,
   25,
   31,

If I had more time, read/learn more about Unit Testing in python!
https://docs.python.org/3/library/unittest.html